In [7]:
import kagglehub

# Download the cats & dogs dataset from Kaggle
path = kagglehub.dataset_download("bhavikjikadara/dog-and-cat-classification-dataset")
print("Path to dataset files:", path)


Path to dataset files: C:\Users\ifjko\.cache\kagglehub\datasets\bhavikjikadara\dog-and-cat-classification-dataset\versions\1


In [8]:
import os, glob
import numpy as np
from PIL import Image

def _require_power_of_two(n):
    if n < 2 or (n & (n - 1)) != 0:
        raise ValueError(f"n_qubits must be a power of two >= 2, got {n}")

def _img_to_features(path, n_qubits):
    with Image.open(path) as im:
        im = im.convert("L")  # grayscale
        im = im.resize((n_qubits, 1), Image.BILINEAR)
        v = np.asarray(im, dtype=np.float32).reshape(-1)
    return (v / 255.0) * np.pi  # map to [0, π]

def _find_class_dirs(root, class_aliases):
    matches = []
    aliases = {a.lower() for a in class_aliases}
    for dirpath, _, _ in os.walk(root):
        leaf = os.path.basename(dirpath).lower()
        if leaf in aliases:
            for ext in ("*.jpg","*.jpeg","*.png","*.bmp","*.gif","*.webp","*.tif","*.tiff"):
                if glob.glob(os.path.join(dirpath, ext)):
                    matches.append(dirpath)
                    break
    return matches

def _collect_images(folder):
    files = []
    for ext in ("*.jpg","*.jpeg","*.png","*.bmp","*.gif","*.webp","*.tif","*.tiff"):
        files += glob.glob(os.path.join(folder, ext))
    return sorted(files)

def load_cats_dogs_auto(root_dir, n_qubits, limit_per_class=None, shuffle=True):
    _require_power_of_two(n_qubits)
    cat_dirs = _find_class_dirs(root_dir, {"cat","cats"})
    dog_dirs = _find_class_dirs(root_dir, {"dog","dogs"})
    if not cat_dirs or not dog_dirs:
        raise FileNotFoundError(f"Couldn’t locate 'cats' and 'dogs' folders under: {root_dir}")

    cat_files, dog_files = [], []
    for d in cat_dirs: cat_files += _collect_images(d)
    for d in dog_dirs: dog_files += _collect_images(d)

    if limit_per_class:
        cat_files = cat_files[:limit_per_class]
        dog_files = dog_files[:limit_per_class]

    images, labels = [], []
    for p in cat_files:
        try: images.append(_img_to_features(p, n_qubits)); labels.append(-1)
        except: pass
    for p in dog_files:
        try: images.append(_img_to_features(p, n_qubits)); labels.append(+1)
        except: pass

    if shuffle:
        rng = np.random.default_rng(42)
        idx = np.arange(len(images)); rng.shuffle(idx)
        images = [images[i] for i in idx]
        labels = [int(labels[i]) for i in idx]

    return images, labels


In [9]:
from qiskit import QuantumCircuit
from qiskit.circuit import ParameterVector
from qiskit.circuit.library import ZFeatureMap
from qiskit.quantum_info import SparsePauliOp
from qiskit.primitives import StatevectorEstimator as Estimator
from qiskit_machine_learning.neural_networks import EstimatorQNN

def conv_circuit(params):
    qc = QuantumCircuit(2)
    qc.rz(-np.pi / 2, 1)
    qc.cx(1, 0)
    qc.rz(params[0], 0)
    qc.ry(params[1], 1)
    qc.cx(0, 1)
    qc.ry(params[2], 1)
    qc.cx(1, 0)
    qc.rz(np.pi / 2, 0)
    return qc

def conv_layer(num_qubits, prefix):
    qc = QuantumCircuit(num_qubits, name="Conv")
    params = ParameterVector(prefix, length=num_qubits * 3)
    idx = 0
    for q1, q2 in zip(range(0,num_qubits,2), range(1,num_qubits,2)):
        qc = qc.compose(conv_circuit(params[idx:idx+3]), [q1,q2]); qc.barrier(); idx += 3
    for q1, q2 in zip(range(1,num_qubits,2), list(range(2,num_qubits,2))+[0]):
        qc = qc.compose(conv_circuit(params[idx:idx+3]), [q1,q2]); qc.barrier(); idx += 3
    out = QuantumCircuit(num_qubits)
    out.append(qc.to_instruction(), range(num_qubits))
    return out

def pool_circuit(params):
    qc = QuantumCircuit(2)
    qc.rz(-np.pi/2,1); qc.cx(1,0)
    qc.rz(params[0],0); qc.ry(params[1],1)
    qc.cx(0,1); qc.ry(params[2],1)
    return qc

def pool_layer(src, sink, prefix):
    n = len(src)+len(sink)
    qc = QuantumCircuit(n, name="Pool")
    params = ParameterVector(prefix, length=(n//2)*3)
    idx=0
    for s,t in zip(src,sink):
        qc = qc.compose(pool_circuit(params[idx:idx+3]), [s,t]); qc.barrier(); idx+=3
    out = QuantumCircuit(n); out.append(qc.to_instruction(), range(n))
    return out

def build_qcnn(estimator, n_qubits):
    fm = ZFeatureMap(n_qubits)
    ansatz = QuantumCircuit(n_qubits, name="Ansatz")
    active = list(range(n_qubits)); stage=1
    while len(active)>1:
        ansatz.compose(conv_layer(len(active), f"c{stage}"), active, inplace=True)
        half=len(active)//2
        src=list(range(half)); sink=list(range(half,len(active)))
        ansatz.compose(pool_layer(src,sink,f"p{stage}"), active, inplace=True)
        active=active[half:]; stage+=1
    readout=active[0]
    obs=["I"]*n_qubits; obs[readout]="Z"
    observable=SparsePauliOp.from_list([("".join(obs),1)])
    circuit=QuantumCircuit(n_qubits)
    circuit.compose(fm, range(n_qubits), inplace=True)
    circuit.compose(ansatz, range(n_qubits), inplace=True)
    return EstimatorQNN(circuit=circuit.decompose(),
                        observables=observable,
                        input_params=fm.parameters,
                        weight_params=ansatz.parameters,
                        estimator=estimator)


In [10]:
from qiskit_machine_learning.algorithms.classifiers import NeuralNetworkClassifier
from qiskit_machine_learning.optimizers import COBYLA
from sklearn.model_selection import train_test_split

def run_trial(seed, dataset, n_qubits=8, maxiter=50, optimizer_kind="COBYLA"):
    np.random.seed(seed)
    images, labels = dataset
    x_tr, x_te, y_tr, y_te = train_test_split(
        images, labels, test_size=0.3, random_state=seed, stratify=labels
    )
    qnn = build_qcnn(Estimator(), n_qubits)
    init = np.random.uniform(-np.pi, np.pi, qnn.num_weights)
    optimizer = COBYLA(maxiter=maxiter)
    clf = NeuralNetworkClassifier(qnn, optimizer=optimizer, initial_point=init)
    clf.fit(np.asarray(x_tr), np.asarray(y_tr))
    return {
        "train_acc": float(clf.score(x_tr,y_tr)),
        "test_acc":  float(clf.score(x_te,y_te))
    }

def run_many_with_dataset(dataset, n_qubits=8, n_trials=3, maxiter=50):
    results=[]
    for i in range(n_trials):
        res=run_trial(1000+i, dataset, n_qubits, maxiter)
        results.append(res)
        print(f"Trial {i+1}: train={res['train_acc']:.3f}, test={res['test_acc']:.3f}")
    return results


In [ ]:
if __name__ == "__main__":
    n_qubits = 16
    dataset = load_cats_dogs_auto(path, n_qubits=n_qubits, limit_per_class=500)
    cats = sum(1 for y in dataset[1] if y==-1)
    dogs = sum(1 for y in dataset[1] if y==+1)
    print(f"Loaded: cats={cats}, dogs={dogs}, total={len(dataset[1])}")

    results = run_many_with_dataset(dataset, n_qubits=n_qubits, n_trials=1, maxiter=200)
    print("Final results:", results)


No gradient function provided, creating a gradient function. If your Estimator requires transpilation, please provide a pass manager.


Loaded: cats=500, dogs=500, total=1000
